In [ ]:
student_name = ["pablo_gonzalez_lazaro", "alberto_lopez_del_amo"]

# TME2 – Tensor Products, Unitaries, and Measurements

In this session, you will practice the basics of quantum formalism with the help of the `ndlist` class.
We will represent qubits as vectors, multi-qubit systems with tensor products, apply unitary operations (quantum gates), and model projective measurements.

You will need some functions you defined in TME1, so make sure you have that code available. You can also refer to the solutions provided.

⚠️ Important: Use only the `ndlist` class we provide. **Do not use NumPy.**

In [11]:
import ndlists as nd

# import TME1_functions as tme1  # Uncomment and change the module name if you have TME1 functions from your previous work or solutions. YOU NEED TO UNCOMMENT THIS LINE TO USE TME1 FUNCTIONS
import TME1_functions_solution as tme1  # Using the provided solutions for TME1 functions

from math import sqrt, pi, cos, sin, tan, exp
from typing import List

# Prerequisites

Currently, the `ndlist` class does **not** behave like NumPy arrays when adding two objects:

```python
import ndlists as nd

a = nd.ndlist([1, 2, 3])
b = nd.ndlist([1, 0, 3])

print(a + b)   # gives [1, 2, 3, 1, 0, 3]  (concatenation, not addition!)
print(2 * a)   # gives [1, 2, 3, 1, 2, 3]  (concatenation, not multiplication!)
```

During the previous TME, you implemented a function for the multiplication of an `ndlist` object and a scalar.
From now on, you will be able to operate with `ndlist` objects as you would with NumPy arrays when it comes to addition and scalar multiplication.

To enable this behavior, we have modified the `ndlist` class to include the (`__add__`;`__radd__`) methods for addition, `__rmul__` methods for scalar multiplication, (`__sub__`;`__rsub__`) for scalar subtraction, and the division methods `__rtruediv__`. Only multiplication between two `ndlist` objects is not defined (it is your responsibility to use the correct function from TME1 for multiplication).
You can find the updated `ndlist` class in the `ndlists.py` file (make sure to use this updated version). Be aware that the multiplication of two `ndlist` objects will give an element wise multiplication:

```python
import ndlists as nd
a = nd.ndlist([1, 2, 3])
b = nd.ndlist([1, 0, 3])
print(a + b)   # gives [2, 2, 6]  (element wise addition)
print(2 * a)   # gives [2, 4, 6]  (scalar multiplication)
```

Thus, when performing matrix multiplication, make sure to use the correct function from TME1! You have been warned 0_0 !

## 1. States of Multipartite Systems

A single qubit is described by a vector in $\mathbb{C}^2$.
For example, the basis states are:

$$|0\rangle = \begin{bmatrix}1 \\ 0\end{bmatrix} \quad , \quad |1\rangle = \begin{bmatrix}0 \\ 1\end{bmatrix}$$

For multiple qubits, we use **tensor products**.
For instance:

$$|0\rangle \otimes |1\rangle = |01\rangle = \begin{bmatrix}0 \\ 1 \\ 0 \\ 0\end{bmatrix}$$

### Exercise 1: Tensor Products

Implement the function `tensor_product` that computes the tensor product of two vectors represented as `ndlist` objects.

In [ ]:
def tensor_product(A: nd.ndlist, B: nd.ndlist) -> nd.ndlist:
    """
    Compute the tensor product of two ndlists recursively.
    :param A:
    :param B:
    :return: The tensor product A ⊗ B as a ndlist.
    """
    tensor_product = tme1._zeros(A.shape[0] * B.shape[0], A.shape[1] * B.shape[1])
    for j in range(A.shape[0]):
        for k in range(B.shape[0]):
                for l in range(A.shape[1]):
                     for m in range(B.shape[1]):
                        tensor_product[j*B.shape[0]+k][l*B.shape[1]+m] = A[j][l] * B[k][m] 
    return tensor_product

In [58]:
# Test the tensor_product function
v0 = tme1._ket([1, 0])  # |0>
v1 = tme1._ket([0, 1])  # |1>

tp = tensor_product(v0, v1)  # |0> ⊗ |1>
print("Tensor product |0> ⊗ |1>:\n", tp, tp.shape)

Tensor product |0> ⊗ |1>:
 [[0], [1], [0], [0]] (4, 1)


In [59]:
# Bell states
bell_00 = (1 / sqrt(2)) * (tensor_product(v0, v0) + tensor_product(v1, v1))
bell_01 = (1 / sqrt(2)) * (tensor_product(v0, v1) + tensor_product(v1, v0))
bell_10 = (1 / sqrt(2)) * (tensor_product(v0, v0) - tensor_product(v1, v1))
bell_11 = (1 / sqrt(2)) * (tensor_product(v0, v1) - tensor_product(v1, v0))

print("Bell state |Φ+>:\n", bell_00, bell_00.shape)
print("Bell state |Φ->:\n", bell_10, bell_10.shape)
print("Bell state |Ψ+>:\n", bell_01, bell_01.shape)
print("Bell state |Ψ->:\n", bell_11, bell_11.shape)

Bell state |Φ+>:
 [[0.7071067811865475], [0.0], [0.0], [0.7071067811865475]] (4, 1)
Bell state |Φ->:
 [[0.7071067811865475], [0.0], [0.0], [-0.7071067811865475]] (4, 1)
Bell state |Ψ+>:
 [[0.0], [0.7071067811865475], [0.7071067811865475], [0.0]] (4, 1)
Bell state |Ψ->:
 [[0.0], [0.7071067811865475], [-0.7071067811865475], [0.0]] (4, 1)


In [60]:
# Verify that the tensor product also holds for quantum gates operators.
I = nd.ndlist([[1, 0],
            [0, 1]])
X = nd.ndlist([[0, 1],
            [1, 0]])
Y = nd.ndlist([[0, -1j],
            [1j, 0]])
Z = nd.ndlist([[1, 0],
            [0, -1]])

In [61]:
# Example: X ⊗ I
X_I = tensor_product(X, I)
print("X ⊗ I:\n", X_I)

# Example: Z ⊗ Y
Z_Y = tensor_product(Z, Y)
print("Z ⊗ Y:\n", Z_Y)

X ⊗ I:
 [[0, 0, 1, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0]]
Z ⊗ Y:
 [[0, -1j, 0, -0j], [1j, 0, 0j, 0], [0, -0j, 0, 1j], [0j, 0, (-0-1j), 0]]


Let's define the CNOT gate using tensor products. The CNOT gate is defined as:

$$\text{CNOT} = \begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0\end{bmatrix}$$

Express CNOT in terms of tensor products and implement it using the `tensor_product` function.

In [62]:
# CNOT gate using tensor products
m0 = tme1._matrix([v0, tme1._zeros(2, 1)])  # |0><0|
m1 = tme1._matrix([tme1._zeros(2, 1), v1])  # |1><1|

print(m0)
print(m1)

CNOT = tensor_product(m0, I) + tensor_product(m1, X)
print("CNOT gate:\n", CNOT, CNOT.shape)

[[1, 0], [0, 0]]
[[0, 0], [0, 1]]
CNOT gate:
 [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]] (4, 4)


Compute the action of the CNOT gate on the state $|+\rangle \otimes |0\rangle$, where $|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$.

In [63]:
# Define |+> state
v_plus = 1/sqrt(2) * (v0 + v1) # |+> = (|0> + |1>) / sqrt(2)
print(v_plus, v_plus.shape)
# Initial state |+> ⊗ |0>
initial_state = tensor_product(v_plus, v0)
print("Initial state |+> ⊗ |0>:\n", initial_state, initial_state.shape)

# Apply CNOT gate
final_state = tme1._matmul(CNOT, initial_state)
print("Final state after CNOT:\n", final_state, final_state.shape)

[[0.7071067811865475], [0.7071067811865475]] (2, 1)
Initial state |+> ⊗ |0>:
 [[0.7071067811865475], [0.0], [0.7071067811865475], [0.0]] (4, 1)
Final state after CNOT:
 [[0.7071067811865475], [0.0], [0.0], [0.7071067811865475]] (4, 1)


## 2. Projective Measurements

A projective measurement is described by a set of projection operators $\{\Pi_i\}_i$ that satisfy the completeness relation:

- $\sum_i \Pi_i = I$
- $\forall i, \Pi_i^2 = \Pi_i$ ($\Pi_i$ is a projector)

When measuring a state $|\psi\rangle$, the probability of obtaining the outcome associated with $\Pi_i$ is given by:

$$ p(i) = \| \Pi_i |\psi\rangle \|^2 = | \langle \psi | \Pi_i | \psi \rangle | $$

After the measurement, the state collapses to:

$$ |\psi_i\rangle = \frac{\Pi_i |\psi\rangle}{\| \Pi_i |\psi \rangle \|} $$

On paper, computing probabilities in multi-qubit systems can quickly become very tedious.

With code, however, we can explore interesting cases like **measuring Bell states in different bases**.

### Exercise 2.1 – Computational basis projectors

1. Define the projectors $P_0 = |0\rangle\langle 0|$ and $P_1 = |1\rangle\langle 1|$.
2. Build two-qubit projectors $P_{00}, P_{01}, P_{10}, P_{11}$ using tensor products.


In [64]:
def projector(state: nd.ndlist) -> nd.ndlist:
    """
    Construct the projector |ψ><ψ| for a normalized state vector ψ.
    """
    projector = tme1._matmul(state , tme1.bra(state))
    return projector

In [65]:
# Define single-qubit projectors
P0 = projector(v0)  # |0><0|
P1 = projector(v1)  # |1><1|
print("Projector P0:\n", P0)
print("Projector P1:\n", P1)

Projector P0:
 [[(1+0j), 0j], [0j, 0j]]
Projector P1:
 [[0j, 0j], [0j, (1+0j)]]


### Exercise 2.2 – Measurement probabilities in computational basis

Compute the probabilities of measuring the Bell state

$$
|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)
$$

in the computational basis.


In [66]:
def measurement_probability(state: nd.ndlist, projector: nd.ndlist) -> float:
    """
    Compute probability of obtaining outcome associated with projector
    when measuring state.
    """
    return (tme1._matmul(tme1._matmul(tme1.bra(state), projector), state))[0][0].real

In [67]:
# Bell state |Φ+> was defined earlier as bell_00
# Define two-qubit projectors
P00 = tensor_product(P0, P0)  # P0 ⊗ P0
P01 = tensor_product(P0, P1)  # P0 ⊗ P1
P10 = tensor_product(P1, P0)  # P1 ⊗ P0
P11 = tensor_product(P1, P1)  # P1 ⊗ P1
print("Projector P00:\n", P00)
print("Projector P01:\n", P01)
print("Projector P10:\n", P10)
print("Projector P11:\n", P11)

Projector P00:
 [[(1+0j), 0j, 0j, 0j], [0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j]]
Projector P01:
 [[0j, 0j, 0j, 0j], [0j, (1+0j), 0j, 0j], [0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j]]
Projector P10:
 [[0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j], [0j, 0j, (1+0j), 0j], [0j, 0j, 0j, 0j]]
Projector P11:
 [[0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j], [0j, 0j, 0j, (1+0j)]]


In [68]:
# Compute probabilities for Bell state |Φ+>
prob_00 = measurement_probability(bell_00, P00)
prob_01 = measurement_probability(bell_00, P01)
prob_10 = measurement_probability(bell_00, P10)
prob_11 = measurement_probability(bell_00, P11)
print("Probabilities of measuring |Φ+> in computational basis:")
print(f"P(00) = {prob_00}, P(01) = {prob_01}, P(10) = {prob_10}, P(11) = {prob_11}")

Probabilities of measuring |Φ+> in computational basis:
P(00) = 0.4999999999999999, P(01) = 0.0, P(10) = 0.0, P(11) = 0.4999999999999999


### Exercise 2.3 – Projectors in the Hadamard (X) basis

Define the states:

- $|+\rangle = (|0\rangle + |1\rangle)/\sqrt{2}$
- $|-\rangle = (|0\rangle - |1\rangle)/\sqrt{2}$

1. Construct the projectors $P_+ = |+\rangle\langle +|$ and $P_- = |-\rangle\langle -|$.
2. Extend them to two-qubit projectors ($P_{++}, P_{+-}, P_{-+}, P_{--}$).

In [69]:
# (Re)define |+> and |-> states
v_plus = 1/sqrt(2) * (v0+v1)  # |+> = (|0> + |1>) / sqrt(2)
v_minus = 1/sqrt(2) * (v0-v1)   # |-> = (|0> - |1>) / sqrt(2)

In [70]:
# Define single-qubit projectors in the Hadamard basis
P_plus = projector(v_plus)  # |+><+|
P_minus = projector(v_minus)  # |-><-|
print("Projector P_plus:\n", P_plus)
print("Projector P_minus:\n", P_minus)

Projector P_plus:
 [[(0.4999999999999999+0j), (0.4999999999999999+0j)], [(0.4999999999999999+0j), (0.4999999999999999+0j)]]
Projector P_minus:
 [[(0.4999999999999999+0j), (-0.4999999999999999+0j)], [(-0.4999999999999999+0j), (0.4999999999999999+0j)]]


In [71]:
# Extend to two-qubit projectors using tensor_product
P_pp = tensor_product(P_plus, P_plus)  # P_plus ⊗ P_plus
P_pm = tensor_product(P_plus, P_minus)  # P_plus ⊗ P_minus
P_mp = tensor_product(P_minus, P_plus)  # P_minus ⊗ P_plus
P_mm = tensor_product(P_minus, P_minus)  # P_minus ⊗ P_minus

print("Projector P_pp:\n", P_pp)
print("Projector P_pm:\n", P_pm)
print("Projector P_mp:\n", P_mp)
print("Projector P_mm:\n", P_mm)

Projector P_pp:
 [[(0.2499999999999999+0j), (0.2499999999999999+0j), (0.2499999999999999+0j), (0.2499999999999999+0j)], [(0.2499999999999999+0j), (0.2499999999999999+0j), (0.2499999999999999+0j), (0.2499999999999999+0j)], [(0.2499999999999999+0j), (0.2499999999999999+0j), (0.2499999999999999+0j), (0.2499999999999999+0j)], [(0.2499999999999999+0j), (0.2499999999999999+0j), (0.2499999999999999+0j), (0.2499999999999999+0j)]]
Projector P_pm:
 [[(0.2499999999999999+0j), (-0.2499999999999999+0j), (0.2499999999999999+0j), (-0.2499999999999999+0j)], [(-0.2499999999999999+0j), (0.2499999999999999+0j), (-0.2499999999999999+0j), (0.2499999999999999+0j)], [(0.2499999999999999+0j), (-0.2499999999999999+0j), (0.2499999999999999+0j), (-0.2499999999999999+0j)], [(-0.2499999999999999+0j), (0.2499999999999999+0j), (-0.2499999999999999+0j), (0.2499999999999999+0j)]]
Projector P_mp:
 [[(0.2499999999999999+0j), (0.2499999999999999+0j), (-0.2499999999999999+0j), (-0.2499999999999999+0j)], [(0.24999999999999

### Exercise 2.4 – Simulation of repeated measurements

The Hadamard basis is often called the X basis because the states $|+\rangle$ and $|-\rangle$ are eigenstates of the Pauli-X operator.

The states $|0\rangle$ and $|1\rangle$ can be expressed in the Hadamard basis as:

$$ |0\rangle = \frac{1}{\sqrt{2}}(|+\rangle + |-\rangle) \quad , \quad |1\rangle = \frac{1}{\sqrt{2}}(|+\rangle - |-\rangle) $$

Let's simulate measuring the Bell state $|\Phi^+\rangle$ in the Hadamard basis multiple times (e.g., 100 times).

Implement a function `simulate_measurement(state, projectors, n)` that repeats a projective measurement $n$ times, returning a list of outcomes.
Verify empirically that for $|\Phi^+\rangle$, only outcomes 00 and 11 (or ++ and -- in X basis) occur, with approximately equal frequency, what is that frequency?

In [72]:
import random

def simulate_measurement(state: nd.ndlist, projectors: List[nd.ndlist], n: int) -> List[int]:
    """
    Simulate n projective measurements on the given state using the provided projectors.
    Returns a list of measurement outcomes (indices of projectors).
    """
    probabilities = []
    possible_outputs = []
    return_list = []

    for i in range(len(projectors)):
        possible_outputs.append(i)

    for i in projectors:
        probabilities.append(measurement_probability(state, i))
    
    choices = random.choices(possible_outputs, weights=probabilities, k=n)
    for i in possible_outputs:
        return_list.append(choices.count(i))

    return return_list
    

In [74]:
# Simulate measurements of |Φ+> in the Hadamard basis
print(simulate_measurement(bell_00, [P_pp, P_pm, P_mp, P_mm], 100))

[49, 0, 0, 51]
